# Analysis of respiration data from MEG Lab
This notebook walks through an example analysis of respiration data recorded during the MEG lab session. The goal is to examine basic breathing patterns, how to preprocess the breathing signal and explore whether the phase of the respiratory cycle relates to perceptual sensitivity to the visual stimuli presented during the experiment.

Author: [Laura Bock Paulsen](https://github.com/laurabpaulsen)
Date: November 2025



## Setup

First, we install the respiration-analysis package developed for Laura's masters thesis (⚠️ still under active development — let me know if you run into any issues). A preliminary version of the documentation is available [here](laurabpaulsen.github.io/pyriodic/).

We also install two additional packages:

* one for Bayesian estimation of the psychometric function,
* and statsmodels, which we’ll use later for the making linear mixed effects models


In [ ]:
!pip install git+https://github.com/laurabpaulsen/pyriodic.git
!pip install psignifit
!pip install statsmodels

### Importing packages
In this step, we load the core libraries needed for the analysis.

In [ ]:
from pathlib import Path

# --- Data & signal processing ---
import numpy as np
import pandas as pd
import mne

# --- Plotting ---
import matplotlib.pyplot as plt
%matplotlib inline

# --- Respiration analysis (pyriodic) ---
from pyriodic.preproc import RawSignal
from pyriodic.viz import plot_phase_diagnostics, CircPlot
from pyriodic.phase_events import create_phase_events
from pyriodic import Circular
from pyriodic.permutation import permutation_test_between_samples

# --- Psychophysics & statistics ---
import psignifit as ps
import psignifit.psigniplot as psp
import statsmodels.formula.api as smf

### Defining paths and loading data

In this step, we load the raw MEG data and the behavioural log for one participant. 

For today’s analysis, we only need two channels:
* MISC001: the respiration signal
* STI101: the stimulus trigger channel 

🚨 Feel free to explore another participant by changing the `subject` and `session_folder` variables below!

In [ ]:
MEG_path = Path("/work/MEG_data/workshop_data")
subject = "0164"#"0165"
session_folder = "20251003_000000" 
raw_path = MEG_path / subject / session_folder / "workshop_2025_raw.fif"
behav_path = list((MEG_path / "behavioural_logs").glob(f"{subject}*"))[0]

raw = mne.io.read_raw_fif(raw_path)

raw.pick(["MISC001", "STI101"]) # might be MISC004 for some participants!!
raw.load_data()

### Loading logfile and extracting events

Next, we load the behavioural logfile and extract the stimulus events from the MEG data.
The MEG recording contains many event markers, so we filter them to keep only the triggers corresponding to the two stimulus conditions used in the experiment.

We also add a simple correct/incorrect column to the behavioural dataframe so we can later link breathing phase to task performance.


In [ ]:
# Load the logfile
df = pd.read_csv(behav_path, index_col=False)

events = mne.find_events(raw, shortest_event=1)

# Only keep events with trigger values in event_ids (for speed here only including the targets)
event_ids_exp = {
    "stim/0": 1, 
    "stim/1": 3
}

valid_event_values = set(event_ids_exp.values())
events = events[np.isin(events[:, 2], list(valid_event_values))]

df["correct"] = (df["target_type"] == df["objective_response"]).astype(int)

### Adding accuracy to the MEG events
Here we attach the behavioural accuracy (correct = 1, incorrect = 0) to each MEG event so that the event codes reflect trial outcome.

In [ ]:
print("First 5 events before:\n", events[:5], "\n")

event_ids= {
    "/correct": 1,
    "incorrect": 0
}

new_events=[]

for event, correct in zip(events, df["correct"]):
    new_events.append([event[0], event[1], correct])

new_events = np.array(new_events)
print("First 5 updated events:\n", new_events[:5])

## Preprocessing of the respiratory signal
Next, we prepare the respiration signal for analysis. This includes filtering, normalising, and extracting the phase of the respiratory cycle.

First, we extract the respiration channel from the raw mne object. We then create a `pyriodic.RawSignal` object, which provides methods for preprocessing and visualisation.

In [ ]:
# extract data from the mne Raw object
resp_ts = raw.get_data(picks=["MISC001"]).squeeze()

# Create a pyriodic object with the respiration data
resp_raw = RawSignal(resp_ts, fs=raw.info["sfreq"])
resp_raw.plot(start=100, duration=30)

### Filtering
As with MEG data, we can bandpass filter the respiration signal between 0.1 and 1 Hz (≈6–60 breaths per minute) to retain normal breathing rhythms while removing slow drifts and high-frequency noise.

In [ ]:
resp_raw.filter_bandpass(low=0.1, high=1)
resp_raw.plot(start=100, duration=30)

### Extracting phase angles
To describe the respiration cycle in terms of its position within each breath, we assign a phase angle to every point in the signal.
We use the two-point interpolation method:

* Peak → Trough: linearly interpolates phase from 0 to π
* Trough → Peak: linearly interpolates phase from π to 2π

This way, every moment in the breath cycle has a number between 0 and 2π that reflects whether the person is inhaling or exhaling.

In [ ]:
phase, peaks, troughs = resp_raw.phase_twopoint(
    prominence=0.01,  # play around with these parameters if the peak detection seems to fail in the following plot!
    distance=0.5
    )

# we can also extract the phase angles using other methods like 
# one-point which interpolates between 0 and 2pi from peak to peak
phase_onepoint, peaks_onepoint = resp_raw.phase_onepoint(
    prominence=0.01,  
    distance=0.5
    )

#### Plotting the extracted phase and respiration signal to check quality


In [ ]:
%matplotlib widget 

plot_phase_diagnostics(
    {
        "Two-point": phase,
        "One-point": phase_onepoint
    },
    start = 500,
    window_duration = 35,
    fs = resp_raw.fs,
    data = resp_raw.ts, #the preproccessed data
    peaks=peaks,
    troughs=troughs,
)

### Distribution of phase angles across the respiratory cycles

In [ ]:
%matplotlib inline 

circ_onepoint_ts = Circular(phase_onepoint)
circ_twopoint_ts = Circular(phase)

fig, axes = plt.subplots(1, 2, dpi=300, subplot_kw={'projection': 'polar'})

for tmp_circ, ax in zip([circ_onepoint_ts, circ_twopoint_ts], axes):
    tmp_plot = CircPlot(tmp_circ, ax=ax)
    tmp_plot.add_histogram()


# make both polar plots share the same radial (y) limit
rmax = max(ax.get_rmax() for ax in axes)
for ax in axes:
    ax.set_rlim(0, rmax+0.3)

<div class="alert alert-block alert-info">
<b>Question:</b> What may the benefit be in terms of using more simple statistics from using one-point interpolation?
</div>

**Answer:**

<div class="alert alert-block alert-info">
<b>Question:</b> Imagine a mean phase angle of target events is π. What would the interpretation of this result be using the two-point and one-point interpolation methods, respectively? I.e., what can you say about where participants tend to be in their breathing cycle when targets occur?
</div>

**Answer:**

### Extracting phase events
We will now continue using the phase angles extracted using the two-point method. Now that we have phase angles for the time series respiration signal, we can link each stimulus event to the corresponding breathing phase.

In [ ]:
trigger_vals = new_events[:, 2] # the updated trigger values with accuracy info
sample_indices = new_events[:, 0] # the sample indices of each event

label_mapping = {val: key for key, val in event_ids.items()} # reverse the event_ids dict
event_labels = np.array([label_mapping.get(trig, "unknown") for trig in trigger_vals])


circ = create_phase_events(
    phase_ts=phase,
    events=sample_indices,
    event_labels=event_labels,
    first_samp = raw.first_samp, # account for MEG first sample offset between the time series and the events
    rejection_method = 'segment_duration_sd', # excludes events during inspiration/exiration segments whose durations deviate more than `rejection_criterion` standard deviations from the mean
    rejection_criterion = 3
)

### Initial plot of the distribution of respiratory phase angles for the events


In [ ]:
plot = CircPlot(circ, group_by_labels=True, colours=["forestgreen", "darkorange"])
plot.add_points(s=1, alpha=0.5)
plot.add_histogram(phase)
plot.add_density()
plot.add_legend()

## Do correct vs. incorrect trials differ in respiratory phase?

To formally test whether correct and incorrect trials tend to occur at different points in the respiratory cycle, we use a permutation-based comparison of circular data. This approach makes no assumptions about the underlying distribution and is therefore appropriate for phase angles, which rarely meet parametric criteria.

If correct and incorrect trials come from the same underlying phase distribution, then shuffling their labels should not change the test statistic.

**Watson’s $U^2$ test statistic**

By default, the permutation framework uses Watson’s $U^2$. This is a standard circular statistic that measures how much two cumulative phase distributions differ. Larger values indicate larger differences between conditions.

**How the permutation test works**

Internally, the test proceeds as:

* Compute the observed statistic (e.g., Watson’s $U^2$) between the actual correct and incorrect samples
* Combine and shuffle the samples repeatedly, each time recomputing the statistic to build a null distribution that represents the differences we would expect to observe by chance
* Calculate the p-value as the fraction of permuted statistics that exceed the observed one (setting alternative="greater")

This tells us whether the observed distributional differences between correct and incorrect phase angles is larger than expected by chance.

In [ ]:
obs_stat, pval = permutation_test_between_samples(
    circ["/correct"].data,
    circ["incorrect"].data,
    n_permutations=10000,
    # we want to know whether the distributional differences between the observed groups are larger than expected by chance
    alternative="greater" 
)

<div class="alert alert-block alert-info">
<b>Question:</b> INSERT QUESTION HERE!
</div>

## Do sensitivity differ systematically across the respiratory cycle?

Next, we ask whether a participant’s perceptual sensitivity depends on the phase of their breathing. To do this, we take the following steps:


1. Fit a psychometric function
    * First, we fit a cumulative Gaussian psychometric function to all trials for the participant using the `psignifit` toolbox.
    * This gives a baseline estimate of the participant’s threshold and other parameters.

2. Moving-window analysis across respiratory phase
    * We divide the respiration cycle (0 → 2π) into 40 overlapping bins, each covering π/4 of the cycle, stepping by π/20.
    * For each bin, we refit the psychometric function keeping all parameters fixed except the threshold.
    * Bins with fewer than 30 trials are skipped to ensure reliable estimates.
    * This produces a threshold estimate for every segment of the breathing cycle.

The first two steps are shown in this notebook. If you wanted to quantify whether the modulation was significant at the group-level, the following steps could be a way of doing that.

---

3. Modelling sensitivity modulation
    * Thresholds are modelled as a function of respiration phase using a linear mixed-effects model:
        threshold ~ sin(phase) + cos(phase) + (1 + sin(phase) + cos(phase) | participant)
    * This captures the periodic nature of the breathing cycle while accounting for individual differences.

4. Quantifying the effect
    * The sine and cosine coefficients are combined into a single vector norm, representing the magnitude of phase-dependent modulation.

5. Statistical testing
    * We generate a null distribution by shuffling thresholds 10,000 times and refitting the model each time.
    * The p-value is the proportion of null vector norms ≥ the observed vector norm.



In [ ]:
# PARAMETERS
# for bins refitting of psychometric function
DELTA_CENTER = np.pi/20 # how much are we moving the center of the bin everytime
WIDTH = np.pi/4 # how wide is the bin
N_NULL_LMEM = 10_000 

MIN_TRIALS_PER_BIN = 30 # how many trials would we like as minimum within each bin to include it in the analysis

Next, we define some helper functions to run the analysis.

In [ ]:
def format_data_for_psignifit(intensities, hit_or_miss):
    """
    Formats the data for psignifit by creating a 2D array where each row corresponds to a unique intensity
    and contains the number of hits and total trials for that intensity.

    Args:
        intensities (np.ndarray): Array of stimulus intensities.
        hit_or_miss (np.ndarray): Array of binary responses (1 for hit, 0 for miss).

    Returns:
        np.ndarray: Formatted data array for psignifit.
    """
    unique_intensities = np.unique(intensities)
    formatted_data = np.zeros((len(unique_intensities), 3))

    for i, intensity in enumerate(unique_intensities):
        hits = np.sum((intensities == intensity) & (hit_or_miss == 1))
        misses = np.sum((intensities == intensity) & (hit_or_miss == 0))
        formatted_data[i] = [intensity, hits, hits + misses]

    return formatted_data



def phase_bin_mask(phase_angles, center, width):
    diff = (phase_angles - center + np.pi) % (2*np.pi) - np.pi
    return np.abs(diff) <= width / 2


In [ ]:
# empty dataframe to store threshold estimates

PA_correct = circ["/correct"].data
PA_incorrect = circ["incorrect"].data
contrast = df["target_contrast"].round(2) # rounding to fewer decimals to save fitting time

idx_correct = [idx for idx, label in enumerate(circ.labels) if label =="/correct"]
idx_incorrect = [idx for idx, label in enumerate(circ.labels) if label =="incorrect"]

contrast_correct, contrast_incorrect = contrast[idx_correct], contrast[idx_incorrect]

correct_or_incorrect = np.concatenate([
                np.ones(len(PA_correct), dtype=int),
                np.zeros(len(PA_incorrect), dtype=int)
        ])
contrast = np.concatenate([contrast_correct, contrast_incorrect])


PA = np.concatenate([PA_correct, PA_incorrect])


psignifit_kwargs = {
        'experiment_type': '2AFC', 
        'stimulus_range': [np.min(contrast), np.max(contrast)],
    }

psignifit_data = format_data_for_psignifit(contrast, correct_or_incorrect)

result_all_data = ps.psignifit(
        psignifit_data, **psignifit_kwargs
)


### Plot the overall fit!

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

psp.plot_psychometric_function(
    result_all_data,
    ax=ax,
    plot_parameter=True,
    x_label="Contrast"
)
ax.set_ylim(0, 1)

plt.show()

Before moving on, discuss the following questions:
1. What do PA_correct and PA_incorrect represent, and how are they being selected from the circular events?
2. Print `psignifit_data`. What does each of the columns in the array represent?
3. What is the threshold?

### Refitting of the psychometric function using data from different bins of the respiratory cycle
Now to the refitting. Make sure to look through each step of the code to understand what is going on. 

In [ ]:
center_of_bins_loop = np.arange(0, 2 * np.pi, DELTA_CENTER)

refitted_results = []
center_of_bins = []
for i, c in enumerate(center_of_bins_loop):
    mask = phase_bin_mask(PA, c, WIDTH)
    tmp_contrast = contrast[mask]        
    tmp_correct_or_incorrect = correct_or_incorrect[mask]

    if len(tmp_correct_or_incorrect) < MIN_TRIALS_PER_BIN:
        print(f"⚠️ Skipping bin at phase {c:.2f} (only {len(tmp_correct_or_incorrect)} trials)")
        continue
    else:
        center_of_bins.append(c)
    
    # All parameters except the threshold were then fixed and used as priors for fitting the psychometric function iteratively to an angle-specific subset of trials (gray functions).
    tmp_result = ps.psignifit(
        format_data_for_psignifit(tmp_contrast, tmp_correct_or_incorrect),
                
        # fixing parameters from fit on full data set
        fixed_parameters = {
            'lambda': result_all_data.parameter_estimate['lambda'],
            'width': result_all_data.parameter_estimate['width']
        },
        **psignifit_kwargs
    )
    refitted_results.append(tmp_result)

thresholds_refitted = [res.parameter_estimate['threshold'] for res in refitted_results]
zscored_thresholds = (thresholds_refitted - np.mean(thresholds_refitted)) / np.std(thresholds_refitted)
        
threshold_estimates_refitting = pd.DataFrame({
    "participant": subject,
    "center": center_of_bins,        
    "sin_phase": np.sin(center_of_bins),
    "cos_phase": np.cos(center_of_bins),
    "threshold": thresholds_refitted,
    "zscored_threshold": zscored_thresholds
})

### Plot the psychometric functions refittings!

In [ ]:
def plot_subject_level(refitted_results, full_fitted_result, center_of_bins):   
    fig, ax = plt.subplots(figsize=(10, 6))

    cmap = plt.cm.twilight
    norm = plt.Normalize(vmin=0, vmax=2*np.pi)

    # Plot psychometric functions color-coded by phase
    for res, c in zip(refitted_results, center_of_bins):
        color = cmap(norm(c))
        psp.plot_psychometric_function(
            res,
            ax=ax,
            line_color=color,
            line_width=0.8,
            plot_parameter=False,
            plot_data=False
        )
   
    # Plot the global fit on top
    psp.plot_psychometric_function(
        full_fitted_result,
        ax=ax,
        line_color="k",
        line_width=2,
        plot_parameter=False,
        plot_data=True,
        x_label="Contrast",
        y_label="Proportion of hits"
    )

    ax.set_ylim((0, 1))

    # ---- ADD POLAR COLOR LEGEND ----

    inset_size = 0.25  # relative size of inset
    inset_margin = 0.15  # margin from edges
    inset_ax = fig.add_axes([1 - inset_margin - inset_size, inset_margin, inset_size, inset_size], projection='polar')
    theta = np.linspace(0, 2*np.pi, 200)
    radii = np.ones_like(theta)


    circ = Circular(center_of_bins) 
    plot = CircPlot(circ=circ, group_by_labels=False, ax=inset_ax)
    

    thresholds = [res.parameter_estimate['threshold'] for res in refitted_results]
    thresholds = np.array(thresholds)

    plot.add_points(
        y = thresholds,
        color = [cmap(norm(c)) for c in center_of_bins],
        s=10
    )

    # add hline at the threshold of the full data fit
    plot.add_hline(full_fitted_result.parameter_estimate['threshold'], c = "k", linestyle='--', linewidth=1,)

    inset_ax.set_ylim(np.min(thresholds)*0.7, np.max(thresholds) * 1.3)
    

    return fig, ax

plot_subject_level(refitted_results, result_all_data, center_of_bins)


<div class="alert alert-block alert-info">
<b>Question:</b> Looking at the above plot, is there anyway you could change the experiment to get better estimates of the threshold in each of the phase bins?
</div>

**Answer:**